In [ ]:
import numpy as np

from scipy.interpolate import interp1d
from diffusionpy import D_Matrix,Diffusion_MS_iter,Diffusion_MS,time_dep_surface_cryst,lngi,vpure,dlnai_dlnxi
import matplotlib.pyplot as plt

In [ ]:
crystallize=np.asarray([False,False,True])
mobile=np.asarray([True,False,False])
mobiles=np.where(mobile)[0]
immobiles=np.where(~mobile)[0]
deltaHSL=np.asarray([31500.])
TSL=np.asarray([429.47])
cpSL=np.asarray([87.44])

In [ ]:
data=np.asarray([[0.,	0.27288],
       [0.05   , 0.23445],
       [0.1    , 0.19755],
       [0.15   , 0.17761],
       [0.2    , 0.16723],
       [0.25   , 0.15672],
       [0.3    , 0.14609],
       [0.35   , 0.13535],
       [0.45   , 0.11365],
       [0.5    , 0.10276],
       [0.55   , 0.09191],
       [0.6    , 0.0812 ],
       [0.65   , 0.07075],
       [0.7    , 0.0607 ],
       [0.75   , 0.05123],
       [0.8    , 0.04251],
       [0.85   , 0.03468],
       [0.9    , 0.03224],
       [0.95   , 0.0312 ],
       [1.     , 0.03033]])
ww=data[:,1] #np.asarray([0.27087,0.22302, 0.13792, 0.09208, 0.06118])
dl=data[:,0] #np.asarray([0,0.1 , 0.3 , 0.5 , 0.68])
DAPI=np.asarray([6.6E-17])
sigma=np.asarray([1.98E-02])
kt=np.asarray([5.1E-12])
g=np.asarray([3.2])
Mi=np.asarray([18.015,65000.,230.26])
rho0i=np.asarray([997.,1180.,1320.])

In [ ]:
wv_fun=interp1d(dl,ww,bounds_error=False,fill_value=(np.max(ww),np.min(ww)))

In [ ]:
nc=3
wv0=0.0001
dl0=0.68
wi0=np.asarray([wv0,(1-wv0)*(1-dl0),(1-wv0)*dl0])
wv8=wv_fun(dl0)
wi8=np.asarray([wv8,(1-wv8)*(1-dl0),(1-wv8)*dl0])
T=298.15
p=1E5

In [ ]:
kij=D_Matrix(np.asarray([-0.128,0.00648,-0.0574]),nc)
par={"mi":np.asarray([1.20469,2420.99, 8.105152]),
"si": np.asarray([2.797059952,2.947, 2.939]),
"ui" :np.asarray([353.95,205.27, 229.45]),
"eAi" :np.asarray([2425.67,0., 934.2]),
"kAi":np.asarray([0.04509,0.02, 0.02]),
"NAi":np.asarray([1.,653., 2.]),
"Mi": Mi,
"kij":kij}

In [ ]:
vpures=vpure(p,T,**par)

In [ ]:
par["vpure"]=vpures
lngi_fun=lambda wi: lngi(T,wi,**par)

In [ ]:
nt=300
t=np.linspace(0,1300,nt)*60
witB,alpha,r=time_dep_surface_cryst(t,mobiles,immobiles,crystallize,wi0,wi8,rho0i,Mi,DAPI,sigma,kt,g,deltaHSL,TSL,cpSL,lngi_fun,wv_fun)
tmin=t/60
# fig1,ax1=plt.subplots()
# fig2,ax2=plt.subplots()
# ax1.plot(tmin,alpha)
# rmu=r/1E-6
# ax2.plot(tmin,rmu)
# import pandas as pd
# pd.DataFrame((tmin,alpha,rmu)).to_clipboard()
# plt.plot(t/60,witB[0,:])
# plt.plot(t/60,witB[1,:])
# plt.plot(t/60,witB[2,:])
Dvec=np.asarray([5E-13,5E-13,5E-13])
L=2.5E-5
dlnai_dlnwi_fun=lambda wi: dlnai_dlnxi(T,wi,**par)
wt=Diffusion_MS(t,L,Dvec,wi0,wi8,Mi,mobile,swelling=True,witB=witB.T)
wt=Diffusion_MS_iter(t,L,Dvec,wi0,wi8,Mi,mobile,swelling=True,witB=witB.T,dlnai_dlnwi_fun=dlnai_dlnwi_fun)
XwL=wt[:,0]/(1-wt[:,0])
Xw=XwL*(1-alpha)
ww=Xw/(1+Xw)
# plt.plot(t/60,witB.T[:,0],"bo")
plt.plot(t/60,Xw,'b',label="Non-ideal and wiB(t)")
plt.legend("")
plt.show()
import pandas as pd
pd.DataFrame((tmin,ww)).T.to_clipboard(excel=True, sep=None, index=False, header=None)